In [4]:
# import libraries
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import pandas as pd
import datetime

client_ID='4e1c1626b9e04c0fba6d8f14d31ab3e6'
client_SECRET='607dd5362f9d4f44b33361eca5aa81b8'   
redirect_uri='http://127.0.0.1:9090'
username = 'rt47etgc6xpwhhhb8575rth83'

# 
# data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'


# retrun spotify token for the given scope
def get_spotify_token(scope):
    client_credentials_manager = spotipy.oauth2.SpotifyOAuth(
        scope=scope,
        username=username,
        client_id=client_ID,
        client_secret=client_SECRET,
        redirect_uri=redirect_uri,
        # open_browser=False
        )
    client_credentials_manager.get_access_token(as_dict=False)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# # token to modify user's playlists
# playlist_modify_token = get_spotify_token("playlist-modify-public")

# # token to read user's library
# library_read_token = get_spotify_token("user-library-read")

# # token to read user's recently played tracks
# recently_played_token = get_spotify_token("user-read-recently-played")

# # token to read user's top tracks
# top_tracks_token = get_spotify_token("user-top-read")

# # token to read user's top artists
# top_artists_token = get_spotify_token("user-top-read")

# # token to read user's playlists
# playlist_read_token = get_spotify_token("playlist-read-private")

# # token to read user's followers
# user_followers_token = get_spotify_token("user-follow-read")


# get the last 50 tracks played by the user
def get_recently_played_tracks(token):
    recently_played = token.current_user_recently_played(limit=50)
    return recently_played

# get users top tracks
def get_top_tracks(token):
    top_tracks = token.current_user_top_tracks(limit=50)
    return top_tracks

def timestamp_to_time(timestamp):
    try:
        # Try to parse the timestamp string as ISO format with microsecond component
        dt_object = datetime.datetime.fromisoformat(timestamp[:-1])
    except ValueError:
        try:
            # Try to parse the timestamp string as ISO format without microsecond component
            dt_object = datetime.datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError:
            # Try to parse the timestamp string as a Unix timestamp in seconds
            dt_object = datetime.datetime.fromtimestamp(int(timestamp))

    return dt_object


# get all the useful information from the recently played tracks
def get_recently_played_tracks_info():
    # list to store the recently played tracks
    recently_played_tracks = []
    # get the recently played tracks
    recently_played_token = get_spotify_token("user-read-recently-played")
    recently_played = get_recently_played_tracks(recently_played_token)
    # iterate through all the recently played tracks
    for i in range(len(recently_played['items'])):
        # get the track info
        track = recently_played['items'][i]['track']
        # create a dictionary to store the track info
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'played_at': recently_played['items'][i]['played_at']
        }
        # print the track info
        song_name = track['name']
        # print the track time and name
        # print(recently_played['items'][i]['played_at'], song_name)

        recently_played_tracks.append(track_info)
    return recently_played_tracks




# function to add the recently played tracks in a csv file
def store_recently_played_tracks():
    # get the recently played tracks
    recently_played_tracks = get_recently_played_tracks_info()
    # create a dataframe from the stored recently played tracks
    old_recently_played_tracks = pd.read_csv(recently_played_file_name)
    # create a dataframe from the list of recently played tracks
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # iterate through all the new_recently_played_tracks and print the track name and time
            for j in range(len(new_recently_played_tracks)):
                print(new_recently_played_tracks['played_at'].iloc[j], new_recently_played_tracks['track_name'].iloc[j])
            # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_name, index=False)
            break




In [5]:
# store the recently played tracks
store_recently_played_tracks()

In [6]:
def get_top_tracks(token):
    top_tracks = token.current_user_top_tracks(limit=50, time_range=time_range)
    return top_tracks

# write function to read user's top tracks
def get_top_tracks_info():
    token = get_spotify_token("user-top-read")
    top_tracks = get_top_tracks(token)
    top_tracks_info = []
    for i in range(len(top_tracks['items'])):
        track = top_tracks['items'][i]
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        top_tracks_info.append(track_info)
    return top_tracks_info




# write function to store user's top tracks
# def store_top_tracks():
#     top_tracks = get_top_tracks_info()
#     top_tracks_file_path = os.path.join(data_folder, top_tracks_file_name)
#     top_tracks_df = pd.DataFrame(top_tracks)
#     top_tracks_df.to_csv(top_tracks_file_path, index=False)
    


In [7]:
print(get_top_tracks_info())

NameError: name 'time_range' is not defined

In [102]:
# location of the folder to store the data



# # get the top 50 tracks played by the user
# def get_top_tracks(token):
#     top_tracks = token.current_user_top_tracks(limit=50)
#     return top_tracks


datetime.datetime(2023, 3, 13, 17, 34, 12, 804000)

In [5]:


store_recently_played_tracks()



NameError: name 'recently_played_token' is not defined

In [103]:



# store_recently_played_tracks(recently_played_tracks)
recently_played_tracks

2023-03-13T20:10:01.748Z The Seer's Tower
2023-03-13T20:06:07.962Z In This Temple As in The Hearts of Man For Whom He Saved The Earth
2023-03-13T20:05:33.176Z Let's hear that string part again, because I don't think they heard it all the way out in Bushnell
2023-03-13T20:04:52.380Z They Are Night Zombies!! They Are Neighbors!! They Have Come Back From the Dead!! Ahhhh!
2023-03-13T19:59:42.582Z The Predatory Wasp of the Palisades Is Out to Get Us!
2023-03-13T19:54:19.241Z A conjunction of drones simulating the way in which Sufjan Stevens has an existential crisis in the Great Godfrey Maze
2023-03-13T19:54:00.410Z Prairie Fire That Wanders About
2023-03-13T19:50:44.936Z The Man of Metropolis Steals Our Hearts
2023-03-13T19:13:15.141Z To The Workers of The Rock River Valley Region, I have an idea concerning your predicament, and it involves an inner tube, bath mats, and 21 able-bodied men
2023-03-13T19:07:11.811Z Chicago
2023-03-13T19:01:07.178Z One last "Whoo-hoo!" for the Pullman
2023-0

[{'track_name': "The Seer's Tower",
  'track_id': '4tcxg3xWf4y0V6mTt1VSFo',
  'track_duration': 233986,
  'track_popularity': 35,
  'track_explicit': False,
  'album_name': 'Illinois',
  'album_id': '1pOl0KEC1iQnA6F0XxV4To',
  'album_release_date': '2005-07-05',
  'album_release_date_precision': 'day',
  'album_total_tracks': 22,
  'album_type': 'album',
  'artist_name': 'Sufjan Stevens',
  'artist_id': '4MXUO7sVCaFgFjoTI5ox5c',
  'played_at': '2023-03-13T20:10:01.748Z'},
 {'track_name': 'In This Temple As in The Hearts of Man For Whom He Saved The Earth',
  'track_id': '4AXlxpofniQO0rJrXuc3U5',
  'track_duration': 35386,
  'track_popularity': 35,
  'track_explicit': False,
  'album_name': 'Illinois',
  'album_id': '1pOl0KEC1iQnA6F0XxV4To',
  'album_release_date': '2005-07-05',
  'album_release_date_precision': 'day',
  'album_total_tracks': 22,
  'album_type': 'album',
  'artist_name': 'Sufjan Stevens',
  'artist_id': '4MXUO7sVCaFgFjoTI5ox5c',
  'played_at': '2023-03-13T20:06:07.962Z'

In [4]:
data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'

# function to add the recently played tracks in a csv file
def store_recently_played_tracks():
    recently_played_tracks = get_recently_played_tracks_info(recently_played)
    # create a dataframe from the list of recently played tracks
    recently_played_file_path = os.path.join(data_folder, recently_played_file_name)
    old_recently_played_tracks = pd.read_csv(recently_played_file_path)
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # print(diff, new_recently_played_tracks['track_name'].iloc[i])
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # print(new_recently_played_tracks['track_name'].iloc[index])
                # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_path, index=False)
            break

store_recently_played_tracks()



NameError: name 'recently_played' is not defined

In [7]:
# asldfkalsdfjasdfj



In [132]:
# # import pydrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive


In [ ]:
# import datetime
import datetime

# get all the useful information from the recently played tracks
def get_recently_played_tracks_info(recently_played):
    recently_played_tracks = []
    recently_played_token = get_spotify_token("user-read-recently-played")
    recently_played = get_recently_played_tracks(recently_played_token)
    for i in range(len(recently_played['items'])):
        track = recently_played['items'][i]['track']
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'played_at': recently_played['items'][i]['played_at']
        }
        # song name
        song_name = track['name']
        print(recently_played['items'][i]['played_at'], song_name)



        recently_played_tracks.append(track_info)
    return recently_played_tracks

# def store_recently_played_tracks():
    recently_played_tracks = get_recently_played_tracks_info(recently_played)
    # create a dataframe from the list of recently played tracks
    # recently_played_file_path = os.path.join(data_folder, recently_played_file_name)
    old_recently_played_tracks = pd.read_csv(recently_played_file_name)
    new_recently_played_tracks = pd.DataFrame(recently_played_tracks)
    # get the last played track from the old_recently_played_tracks
    last_played_at = old_recently_played_tracks['played_at'].iloc[-1]
    # reverse the new_recently_played_tracks
    new_recently_played_tracks = new_recently_played_tracks.iloc[::-1]

    # iterate though all the new_recently_played_tracks and find the one which is closest to the last_played_at
    for i in range(len(new_recently_played_tracks)):
        # get the difference between the last_played_at and the current track's played_at in hours
        diff = (timestamp_to_time(last_played_at) - timestamp_to_time(new_recently_played_tracks['played_at'].iloc[i])).total_seconds() / 3600
        # print(diff, new_recently_played_tracks['track_name'].iloc[i])
        # if the difference is negative, then the current track is the one which was last played
        if diff < 0:
            # get the index of the new_recently_played_tracks
            index = i
            # start the new_recently_played_tracks from the index
            new_recently_played_tracks = new_recently_played_tracks.iloc[index:]
            # print(new_recently_played_tracks['track_name'].iloc[index])
                # add the new_recently_played_tracks to the old_recently_played_tracks, use .concat() to avoid the index being repeated
            old_recently_played_tracks = pd.concat([old_recently_played_tracks, new_recently_played_tracks])
            # save the old_recently_played_tracks to the csv file
            old_recently_played_tracks.to_csv(recently_played_file_path, index=False)
            break


5


In [ ]:
time.time()

1678126686.844148

In [ ]:
# iterate through the first history item and print the track name
for item in history['items']:
    track = item['track']
    print(track['name'])

Oxygen
Hunter
Toxicity
Mother of the World
Screen Shot
Lunacy
The Blacker The Berry
Messenger
Simulation Swarm
Hey Jane
Song Against Sex
She Loves Us
Hacker
She Loves Us
Nosferatu Man
Oxygen
Mladic
Mladic
Mladic
IGOR'S THEME
Flume
Punkero Sonidero
money machine
1539 N. Calvert
Helplessness Blues
Mladic
The Black Hawk War, Or, How to Demolish an Entire Civilization and Still Feel Good About Yourself In the Morning, Or, We Apologize for the Inconvenience But You're Going to Have to Leave Now, Or...
Concerning the UFO sighting near Highland, Illinois
King of Carrot Flowers Pts. 2 & 3
A short reprise for Mary Todd, who went insane, but for very good reasons
Jacksonville
John Wayne Gacy, Jr.
Come On! Feel the Illinoise! Part I: The World's Columbian Exposition Part II: Carl Sandburg Visits Me In A Dream
The Black Hawk War, Or, How to Demolish an Entire Civilization and Still Feel Good About Yourself In the Morning, Or, We Apologize for the Inconvenience But You're Going to Have to Leave Now